In [15]:
import requests #usada para fazer requisições HTTP e fazer buscas no google api
import pandas as pd #usar e manipular dados em forma de tabela = dataframe

In [26]:
generos = ["fiction", "fantasy", "romance", "horror", "mystery"] #lista de generos para a busca
livros_encontrados = [] #lista vazia para adicionar os livros encontrados
for genero in generos:
    url = f"https://www.googleapis.com/books/v1/volumes?q=subject:{genero}&maxResults=10"
    response = requests.get(url)
    data = response.json()

    for x in data.get("items", []): #pega a lista de livros retornada, se sem result [] volta vazio para evitar problemas
        volume_info = x.get("volumeInfo", {}) #Cada livro tem um dicionário chamado "volumeInfo", que contém detalhes como título e autor.
        titulo = volume_info.get("title", "Desconhecido")
        autor = ", ".join(volume_info.get("authors", ["Desconhecido"])) #", ".join(...) converte a lista em uma string separada por vírgulas.

        livros_encontrados.append({"Título": titulo, "Gênero": genero.capitalize(), "Autor": autor})
        # Criamos um dicionário com as informações do livro. genero.capitalize(): Converte o nome do gênero para começar com letra maiúscula.

df = pd.DataFrame(livros_encontrados) # Convertendo a lista dados para um DataFrame Pandas para análise e visualização.
df.head()

,Título,Gênero,Autor
0,Future Shock,Fiction,Alvin Toffler
1,Ready Player Two,Fiction,Ernest Cline
2,Bag of Bones,Fiction,Stephen King
3,Journey to the Center of the Earth,Fiction,Jules Verne
4,How to Stop Time,Fiction,Matt Haig


In [17]:
from jupyter_dash import JupyterDash #roda dash no jupyter
import dash #cria dashboard interativos
from dash import dcc, html #dcc (Dash Core Components): Contém gráficos, dropdowns e outros componentes interativos.
import plotly.express as px #Biblioteca usada para criar gráficos interativos.
from dash.dependencies import Input, Output #criar interatividade no dashboard.

In [18]:
app = dash.Dash(__name__) #cria app usando diretamente o dash, sera respondavel por rodar o dashboard no jupyter

In [19]:
#pegamos a coluna genero do primeiro dataframe e contamos quantos livros tem em cada categ
df_count = df["Gênero"].value_counts().reset_index() #value_counts() conta quantas vezes o genero aparece, reset_index() transforma a contagem em um dataframe novo
df_count.columns = ["Gênero", "Quantidade"] #renomeia as colunas
print(df_count)

    Gênero  Quantidade
0  Fiction          10
1  Fantasy          10
2  Romance          10
3   Horror          10
4  Mystery          10


In [20]:
#layout do dashboar
app.layout = html.Div([
    html.H1("📚 Dashboard de Gêneros Literários Mais Lidos no Brasil", style={'textAlign': 'center'}),

    # Gráfico de barras
    dcc.Graph(id="grafico_generos"),

    # Dropdown para filtrar livros por gênero
    dcc.Dropdown(
        id="filtro_genero",
        options=[{"label": genero, "value": genero} for genero in df["Gênero"].unique()],
        value=df["Gênero"].unique()[0],  # Valor inicial
        clearable=False,
    ),

    # Tabela de livros
    html.Div(id="tabela_livros")
])

In [21]:
dcc.Dropdown( #cria um menu suspenso com as opções de genero
    id="filtro_genero", 
    options =[{"label": genero, "value": genero} for genero in df["Gênero"].unique()], #cria as opções no dropdown
    value=df["Gênero"].unique()[0], #faz o primeiro genero da lista o valor inicial 0
    clearable=False, #faz sempre precisar de um valor
    ),
    #tabela de livros
html.Div(id="tabela_livros") #espaço vazio onde vai ser adicionado uma tabela com os generos escolhidos

Div(id='tabela_livros')

In [22]:
# Callback para atualizar gráfico e tabela
@app.callback(
    Output("grafico_generos", "figure"),
    Output("tabela_livros", "children"),
    Input("filtro_genero", "value")
)
def atualizar_dashboard(genero_selecionado):
    # Atualizar gráfico
    fig = px.bar(df_count, x="Gênero", y="Quantidade", text="Quantidade",
                 title="Livros Mais Lidos por Gênero", color="Gênero")

    # Filtrar tabela pelo gênero selecionado
    df_filtrado = df[df["Gênero"] == genero_selecionado]
    tabela = html.Table([
        html.Thead(html.Tr([html.Th("Título"), html.Th("Autor")])),
        html.Tbody([html.Tr([html.Td(row["Título"]), html.Td(row["Autor"])]) for _, row in df_filtrado.iterrows()])
    ])

    return fig, tabela

# Rodar o app no JupyterLab
app.run_server(mode="inline")

In [23]:
print(df.columns)  # Mostra todas as colunas do DataFrame


Index(['Título', 'Gênero', 'Autor'], dtype='object')


In [24]:
print(df.shape)  # Mostra o número de linhas e colunas
print(df.head())  # Exibe as primeiras linhas do DataFrame


(50, 3)
                               Título   Gênero          Autor
0                        Future Shock  Fiction  Alvin Toffler
1                    Ready Player Two  Fiction   Ernest Cline
2                        Bag of Bones  Fiction   Stephen King
3  Journey to the Center of the Earth  Fiction    Jules Verne
4                    How to Stop Time  Fiction      Matt Haig
